In [1]:
import torch
import sys
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from sklearn import model_selection
from sklearn import metrics
from sklearn.metrics import roc_auc_score, log_loss, roc_auc_score, roc_curve, auc
from sklearn.model_selection import StratifiedKFold, ShuffleSplit, cross_val_score, train_test_split

print('__Python VERSION:', sys.version)
print('__pyTorch VERSION:', torch.__version__)
print('__CUDA VERSION')
from subprocess import call
# call(["nvcc", "--version"]) does not work
! nvcc --version
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())
print('__Devices')
# call(["nvidia-smi", "--format=csv", "--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free"])
print('Active CUDA Device: GPU', torch.cuda.current_device())

print ('Available devices ', torch.cuda.device_count())
print ('Current cuda device ', torch.cuda.current_device())

import numpy
import numpy as np

use_cuda = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
Tensor = FloatTensor

import pandas
import pandas as pd

import logging
handler=logging.basicConfig(level=logging.INFO)
lgr = logging.getLogger(__name__)
%matplotlib inline

# !pip install psutil
import psutil
import os
def cpuStats():
        print(sys.version)
        print(psutil.cpu_percent())
        print(psutil.virtual_memory())  # physical memory usage
        pid = os.getpid()
        py = psutil.Process(pid)
        memoryUse = py.memory_info()[0] / 2. ** 30  # memory use in GB...I think
        print('memory GB:', memoryUse)

cpuStats()

__Python VERSION: 3.7.5 (default, Oct 25 2019, 15:51:11) 
[GCC 7.3.0]
__pyTorch VERSION: 1.2.0
__CUDA VERSION
/bin/sh: 1: nvcc: not found
__CUDNN VERSION: 7301
__Number CUDA Devices: 2
__Devices
Active CUDA Device: GPU 0
Available devices  2
Current cuda device  0
3.7.5 (default, Oct 25 2019, 15:51:11) 
[GCC 7.3.0]
13.5
svmem(total=67477729280, available=44195655680, percent=34.5, used=22047756288, free=18291556352, active=19388362752, inactive=23464185856, buffers=97828864, cached=27040587776, shared=485748736, slab=5039849472)
memory GB: 0.24345779418945312


In [2]:
use_cuda = torch.cuda.is_available()
# use_cuda = False

FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
Tensor = FloatTensor

In [3]:
os.chdir('data')

In [4]:
!pwd

/projects/audio-pipelines/data


In [5]:
# hyps = pd.read_csv("sets/full_set/hyps", names=['audio_id'])
# hyps[['audio_id','hyps']] = hyps["audio_id"].str.split(" ", 1, expand=True)

# targets = pd.read_csv("sets/full_set/targets", names=['audio_id'])
# targets[['audio_id','target']] = targets["audio_id"].str.split(" ", 1, expand=True)

# text = pd.read_csv("sets/full_set/text", names=['audio_id'])
# text[['audio_id','text']] = text["audio_id"].str.split(" ", 1, expand=True)

# speakers = pd.read_csv("sets/full_set/utt2spk", names=['audio_id'])
# speakers[['audio_id','speaker']] = speakers["audio_id"].str.split(" ", 1, expand=True)

# images = pd.read_csv("sets/full_set/wav.scp", names=['audio_id'])
# images[['audio_id','audio_path']] = images["audio_id"].str.split(" ", 1, expand=True)

# dataset = pd.merge(hyps, text, how="left")
# dataset = pd.merge(dataset, speakers, how="left")
# dataset = pd.merge(dataset, images, how="left")
# dataset = pd.merge(dataset, targets, how="left")

In [6]:
# dataset['image_path'] = dataset['audio_path'].apply(lambda x : x[:-4] + ".png")
# def remove_absolute(string, prefix='/home/raznem/projects/audio-pipelines/data/'):
#     if string.startswith(prefix):
#         string = string[len(prefix):]
#     return string
    
# dataset['image_path'] = dataset['image_path'].apply(remove_absolute)

In [7]:
# dataset.head()

In [8]:
dataset = pd.read_csv('data/dataset.csv')

In [9]:
## from allennlp.commands.elmo import ElmoEmbedder
from torchvision import transforms
import numpy as np

try:
    from PIL import Image
except ImportError:
    import Image
    
toImg = transforms.ToPILImage()
toTensor = transforms.ToTensor()
    
class WavImagesLoader(Dataset):
    def __init__(self, csv_path, transform=None):
        self.dataset = pd.read_csv(csv_path)
        self.transform = transform
        
    def __getitem__(self, key):
        target = self.dataset['target'][key]
        image_path = self.dataset['image_path'][key]
        image = Image.open(image_path)
        image = image.convert('RGB')
        image = toTensor(image)
        if self.transform is not None:
            image = self.transform(image)
        
        return image, target
    
    def __len__(self):
        return len(self.dataset)
    
    @staticmethod        
    def imshow(img):
        img = img / 2 + 0.5     # unnormalize
        npimg = img.numpy()
        plt.imshow(np.transpose(npimg, (1, 2, 0)))    

#     @staticmethod    
#     def flaotTensorToImage(img, mean=0, std=1):
#         """convert a tensor to an image"""
#         img = np.transpose(img.numpy(), (1, 2, 0))
#         img = (img*std+ mean)*255
#         img = img.astype(np.uint8)    
#         return img    
    
    
class ElmoWavImagesLoader(WavImagesLoader):
    def __init__(self, csv_path, text_vecs_path, transform=None):
        super().__init__(csv_path, transform=None)
        self.text_vecs_path = text_vecs_path
        
    def __getitem__(self, key):
        target = self.dataset['target'][key]
        image_path = self.dataset['image_path'][key]
        image = Image.open(image_path)
        image = image.convert('RGB')
        image = toTensor(image)
        if self.transform is not None:
            image = self.transform(image)
        
        text_vec = np.zeros((322, 1024))
        tmp = np.load(self.text_vecs_path + self.dataset["audio_id"][key] + ".npy")
        tmp = np.sum(tmp, axis=0)
        text_len = tmp.shape[0]
        text_vec[:tmp.shape[0],:tmp.shape[1]] = tmp
        
        return image, text_vec, text_len, target
    
    
class ElmoWavVecLoader(ElmoWavImagesLoader):
    def __init__(self, csv_path, text_vecs_path, wav_vecs_path, transform=None):
        super().__init__(csv_path, text_vecs_path, transform=None)
        self.wav_vecs_path = wav_vecs_path
        
    def __getitem__(self, key):
        target = self.dataset['target'][key]
        image_path = self.dataset['image_path'][key]
        image_path = image_path[:-4] + ".np"
        wav_vec = np.load(self.wav_vecs_path + image_path, allow_pickle=True)

        text_vec = np.zeros((322, 1024))
        tmp = np.load(self.text_vecs_path + self.dataset["audio_id"][key] + ".npy")
        tmp = np.sum(tmp, axis=0)
        text_len = tmp.shape[0]
        text_vec[:tmp.shape[0],:tmp.shape[1]] = tmp

        return wav_vec, text_vec, text_len, target
    
class ElmoWavVecLoader_v2(WavImagesLoader):
    def __init__(self, csv_path, text_vec_list, wav_vecs_path, transform=None):
        super().__init__(csv_path, transform=None)
        self.wav_vecs_path = wav_vecs_path
        self.text_vec_list = text_vec_list
        
    def __getitem__(self, key):
        target = self.dataset['target'][key]
        image_path = self.dataset['image_path'][key]
        image_path = image_path[:-4] + ".np"
        wav_vec = np.load(self.wav_vecs_path + image_path, allow_pickle=True)

        text_vec = np.zeros((322, 1024))
        tmp = self.text_vec_list[key]
        text_len = tmp.shape[0]
        text_vec[:tmp.shape[0],:tmp.shape[1]] = tmp

        return wav_vec, text_vec, text_len, target

In [10]:
from tqdm import tqdm
files = os.listdir("../text_vecs/")
text_vec_list = []
for i in tqdm(files):
    if not i.endswith(".npy"):
        continue
    tmp = np.load("../text_vecs/" + i)
    tmp = np.sum(tmp, axis=0)
    text_vec_list.append(tmp)

100%|██████████| 27731/27731 [00:17<00:00, 1616.09it/s]


In [11]:
# dataset = pd.read_csv("data/dataset_dropna.csv")

In [12]:
# wav_vec_path = "../wav2vec/"
# wav_vec_list = []
# for i in tqdm(dataset["image_path"]):
#     image_path = i[:-4] + ".np"
#     wav_vec = np.load(wav_vec_path + image_path, allow_pickle=True)
#     break
#     wav_vec_list.append(wav_vec)

Get maximum length of text

In [13]:
# max_text_length = dataset_dropna['hyps'].apply(lambda x: len(x.split(' '))).max()
# max_text_length

## RNN to encode Elmo sequence

In [14]:
class TextWavModel(nn.Module):
    def __init__(self, wav_dim, rnn_dim, layers_rnn=1, text_emb_dim=1024, wav_emb_dim=2115):
        super().__init__()
        self.layers_rnn = layers_rnn
        self.text_emb_dim = text_emb_dim
        self.rnn_dim = rnn_dim
        self.wav_dim = wav_dim
        # Check if weights of both LSTMs are changing
        self.wav_lstm = nn.LSTM(wav_emb_dim, self.wav_dim, num_layers=self.layers_rnn, batch_first=True) 
        self.text_lstm = nn.LSTM(text_emb_dim, self.rnn_dim, num_layers=self.layers_rnn, batch_first=True)
        self.fc_last = nn.Linear(self.wav_dim + self.rnn_dim, 1)
        self.sig = nn.Sigmoid()
        
    def forward(self, wav, text, text_len):
        wav = torch.squeeze(wav)
        _, (wav_out, _) = self.wav_lstm(wav)
        wav_out = wav_out.reshape(-1, self.wav_dim) # check if all dims are ok
        
        seq_lengths = text_len
        pack = torch.nn.utils.rnn.pack_padded_sequence(text, seq_lengths, batch_first=True) # check pack_padded_sequence output
        _, (rnn_out, _) = self.text_lstm(text)
        rnn_out = rnn_out.reshape(-1, self.rnn_dim) # check if all dims are ok
        
        x = torch.cat([wav_out, rnn_out], dim=1)
        y_pred = self.fc_last(x)
        y_pred = torch.squeeze(y_pred)
        y_pred = self.sig(y_pred)
        return y_pred

## Training function

In [15]:
import gc
def wavtext_trainer(loader_train, model, optimizer, exp_name='', save_every=None, print_every=None, epochs=1, use_gpu=True, 
                 dtype=torch.float32):
    if use_gpu and torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu')
        
    model.train()
    model.to(device=device)

    for e in range(epochs):
        print('Epoch %d' %e)
        acc = 0
        files_no = 0
        for t, (wav, text, text_len, y) in enumerate(loader_train):
            wav = wav.to(device=device, dtype=dtype)
            text = text.to(device=device, dtype=dtype)
            y = y.to(device=device, dtype=dtype)
            text_len, perm_idx = text_len.sort(0, descending=True) # check how sort work and it's output
            text = text[perm_idx]
            wav = wav[perm_idx]
            y = y[perm_idx]
            
            y_pred = model(wav, text, text_len)
            
            loss = loss_fn(y_pred, y)
            
            acc += (torch.round(y_pred.cpu()) == y.cpu()).sum().type(torch.float32)
            files_no += len(text)
            optimizer.zero_grad()
            loss.backward()

            optimizer.step()

            if print_every is not None and t % print_every == 0:
                print('Iteration %d, loss = %.4f; epoch %d' % (t, loss.item(), e))
                print(f'Acc {acc / files_no}')
                
                
        if save_every is not None and e % save_every == 0:
            torch.save(model.state_dict(), f'models/{exp_name}_e%d_cnn_rnn.pt' % e)
            gc.collect()
    torch.save(model.state_dict(), f'models/{exp_name}_cnn_rnn.pt')

## Training

In [18]:
BATCH_SIZE = 64
LEARNING_RATE = 1e-4
USE_GPU = True
TE_DTYPE = torch.float32
WAV_DIM = 128
RNN_DIM = 128
NUM_EPOCHS = 100

loss_fn = torch.nn.BCELoss()
model = TextWavModel(WAV_DIM, RNN_DIM)
image_data = ElmoWavVecLoader_v2('data/dataset_dropna.csv', text_vec_list, '../wav2vec/')
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
loader_train = DataLoader(image_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=12)
wavtext_trainer(loader_train, model, optimizer, exp_name='', save_every=10, print_every=100, epochs=NUM_EPOCHS,
      use_gpu=USE_GPU, dtype=TE_DTYPE)

Epoch 0
Iteration 0, loss = 0.6917; epoch 0
Acc 0.546875
Iteration 100, loss = 0.6423; epoch 0
Acc 0.6050432920455933
Iteration 200, loss = 0.6622; epoch 0
Acc 0.6050994992256165
Iteration 300, loss = 0.6831; epoch 0
Acc 0.6066237688064575
Iteration 400, loss = 0.6177; epoch 0
Acc 0.6077384948730469
Epoch 1
Iteration 0, loss = 0.6545; epoch 1
Acc 0.640625
Iteration 100, loss = 0.6508; epoch 1
Acc 0.6047338843345642
Iteration 200, loss = 0.6691; epoch 1
Acc 0.6074315905570984
Iteration 300, loss = 0.6620; epoch 1
Acc 0.6083887219429016
Iteration 400, loss = 0.6759; epoch 1
Acc 0.6088294982910156
Epoch 2
Iteration 0, loss = 0.6625; epoch 2
Acc 0.625
Iteration 100, loss = 0.6924; epoch 2
Acc 0.6141707897186279
Iteration 200, loss = 0.6360; epoch 2
Acc 0.6101523637771606
Iteration 300, loss = 0.7025; epoch 2
Acc 0.607765793800354
Iteration 400, loss = 0.6691; epoch 2
Acc 0.6080501675605774
Epoch 3
Iteration 0, loss = 0.6555; epoch 3
Acc 0.640625
Iteration 100, loss = 0.6756; epoch 3
Acc 0.

KeyboardInterrupt: 